In [107]:
import pandas as pd
import pyreadstat
import statsmodels.api as sm
import numpy as np
from scipy import stats

# 1

In [108]:
path_data = 'econ535_hw1.dta'

df, meta = pyreadstat.read_dta(path_data)
meta.column_labels
meta.variable_value_labels

{}

## Question 1(a)

In [109]:
summary_table_q1a = pd.DataFrame({
    'variable': df.columns,
    'label': meta.column_labels,
    'dtypes': df.dtypes,
    'n_nonnull': df.count(),
    'mean': df.mean(),
})
print(summary_table_q1a)

          variable                                    label   dtypes  \
sampleid  sampleid  unique sample id, links with survey and   object   
quarter    quarter                                     None    int64   
treat        treat                     research group dummy    int64   
higrade    higrade                  highest grade completed   object   
black        black          dummy:ethnicity black (ethnicy)   object   
white        white          dummy:ethnicity white (ethnicy)   object   
otheth      otheth     dummy:ethnicity other (native,asian)   object   
yngchage  yngchage           age yngst child (0-19yrs only)  float64   
adcc          adcc                  quarterly afdc payments    int64   
yr2adc      yr2adc     cova:total afdc payments prior 2 yrs    int64   
yradc        yradc      cova:total afdc payments prior year    int64   
earn          earn                       quarterly earnings    int64   
yrearn      yrearn        cova:earnings in year prior to ra    i

In [110]:
print(df.shape)

(11260, 24)


For earnings, welfare income, total income, education, age, and age squared, the output shows complete responses except for education, which has fewer non-missing observations.

## Question 1(b)

In [111]:
for outcome_q1b in ['earn', 'adcc', 'tinc']:
    y_q1b = df[outcome_q1b]
    X_q1b = pd.DataFrame({'const': 1.0}, index=df.index)

    model_q1b = sm.OLS(y_q1b, X_q1b, missing='drop').fit()
    print('\\n', outcome_q1b)
    print('N used:', int(model_q1b.nobs))
    print('Intercept (regression):', float(model_q1b.params['const']))
    print('Mean (dropna):', float(y_q1b.dropna().mean()))

\n earn
N used: 11260
Intercept (regression): 571.5408525754883
Mean (dropna): 571.5408525754884
\n adcc
N used: 11260
Intercept (regression): 543.4680284191828
Mean (dropna): 543.4680284191829
\n tinc
N used: 11260
Intercept (regression): 1691.3188277087029
Mean (dropna): 1691.3188277087033


The intercept from each constant-only regression equals the corresponding sample mean, which is exactly the relationship requested in Question 1(b).

## Question 1(c)(i)(A)

In [112]:
numeric_cols_q1c = ['higrade', 'age', 'agesq']
df[numeric_cols_q1c] = df[numeric_cols_q1c].apply(pd.to_numeric, errors='raise').astype(float)

analysis_df_q1q2 = df[['earn', 'higrade', 'age', 'agesq']].dropna()

In [113]:
y_q1c_iA_main = analysis_df_q1q2['earn']
X_q1c_iA_main = analysis_df_q1q2[['higrade', 'age', 'agesq']]
X_q1c_iA_main = sm.add_constant(X_q1c_iA_main)

model_q1c_iA_main = sm.OLS(y_q1c_iA_main, X_q1c_iA_main).fit()
print(model_q1c_iA_main.summary())

                            OLS Regression Results                            
Dep. Variable:                   earn   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     102.5
Date:                Mon, 16 Feb 2026   Prob (F-statistic):           2.08e-65
Time:                        16:23:03   Log-Likelihood:                -91983.
No. Observations:               10940   AIC:                         1.840e+05
Df Residuals:                   10936   BIC:                         1.840e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1221.0501    186.247     -6.556      0.0

In [114]:
X_q1c_iA_age = sm.add_constant(analysis_df_q1q2[['age', 'agesq']])
earn_resid_q1c_iA = sm.OLS(analysis_df_q1q2['earn'], X_q1c_iA_age).fit().resid

In [115]:
higrade_resid_q1c_iA = sm.OLS(analysis_df_q1q2['higrade'], X_q1c_iA_age).fit().resid.rename('higrade')

In [116]:
y_q1c_iA_fwl = earn_resid_q1c_iA
X_q1c_iA_fwl = sm.add_constant(higrade_resid_q1c_iA)

model_q1c_iA_fwl = sm.OLS(y_q1c_iA_fwl, X_q1c_iA_fwl).fit()
print(model_q1c_iA_fwl.summary())

print('Education coefficient (main model):', model_q1c_iA_main.params['higrade'])
print('Education coefficient (FWL with constant):', model_q1c_iA_fwl.params['higrade'])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     256.3
Date:                Mon, 16 Feb 2026   Prob (F-statistic):           4.98e-57
Time:                        16:23:03   Log-Likelihood:                -91983.
No. Observations:               10940   AIC:                         1.840e+05
Df Residuals:                   10938   BIC:                         1.840e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -7.844e-12     10.371  -7.56e-13      1.0

The education coefficient matches across the full regression and the residual-on-residual regression, verifying the Frisch-Waugh-Lovell result requested in 1(c)(i)(A).

## Question 1(c)(i)(B)

In [117]:
y_q1c_iB = earn_resid_q1c_iA
X_q1c_iB = higrade_resid_q1c_iA

model_q1c_iB = sm.OLS(y_q1c_iB, X_q1c_iB).fit()
print(model_q1c_iB.summary())

print('Education coefficient (FWL with constant):', model_q1c_iA_fwl.params['higrade'])
print('Education coefficient (FWL no constant):', model_q1c_iB.params['higrade'])

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.023
Model:                            OLS   Adj. R-squared (uncentered):              0.023
Method:                 Least Squares   F-statistic:                              256.3
Date:                Mon, 16 Feb 2026   Prob (F-statistic):                    4.92e-57
Time:                        16:23:03   Log-Likelihood:                         -91983.
No. Observations:               10940   AIC:                                  1.840e+05
Df Residuals:                   10939   BIC:                                  1.840e+05
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

The residualized regressor has mean zero, so adding a constant does not change the slope estimate in this residual-on-residual regression.

## Question 1(c)(ii)

In [118]:
demeaned_df_q1c_ii = analysis_df_q1q2 - analysis_df_q1q2.mean()

y_q1c_ii = demeaned_df_q1c_ii['earn']
X_q1c_ii = demeaned_df_q1c_ii[['higrade', 'age', 'agesq']].to_numpy()

model_q1c_ii = sm.OLS(y_q1c_ii, X_q1c_ii).fit()
residuals_q1c_ii = model_q1c_ii.resid.to_numpy()
fitted_q1c_ii = model_q1c_ii.fittedvalues.to_numpy()
print(model_q1c_ii.summary())

print('Main slopes:')
print(model_q1c_iA_main.params[['higrade', 'age', 'agesq']])
print('Demeaned slopes:')
print(pd.Series(model_q1c_ii.params, index=['higrade', 'age', 'agesq']))

                                 OLS Regression Results                                
Dep. Variable:                   earn   R-squared (uncentered):                   0.027
Model:                            OLS   Adj. R-squared (uncentered):              0.027
Method:                 Least Squares   F-statistic:                              102.5
Date:                Mon, 16 Feb 2026   Prob (F-statistic):                    2.05e-65
Time:                        16:23:03   Log-Likelihood:                         -91983.
No. Observations:               10940   AIC:                                  1.840e+05
Df Residuals:                   10937   BIC:                                  1.840e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

The demeaned no-constant regression reproduces the same slope coefficients as the original regression with a constant.

## Question 1(d)

In [119]:
print(np.round(residuals_q1c_ii.sum(), 4))

0.0


In [120]:
print(np.round(y_q1c_ii.mean(), 4))
print(np.round(fitted_q1c_ii.mean(), 4))

0.0
-0.0


In [121]:
Xt_resid_q1d = X_q1c_ii.T @ residuals_q1c_ii
print("X' u =", np.round(Xt_resid_q1d, 4))

X' u = [ 0. -0. -0.]


In [122]:
corr_q1d = np.corrcoef(y_q1c_ii, fitted_q1c_ii)[0, 1]
r2_from_corr_q1d = corr_q1d**2

print('corr(y,yhat)^2:', r2_from_corr_q1d)
print('R^2 (statsmodels):', model_q1c_ii.rsquared)

corr(y,yhat)^2: 0.027337872608211338
R^2 (statsmodels): 0.02733787260821141


The four printed checks verify each requested OLS property in Question 1(d).

## Question 1(e)

In [123]:
y_q1e = earn_resid_q1c_iA.to_numpy()
X_q1e = higrade_resid_q1c_iA.to_numpy()

residuals_q1e = model_q1c_iB.resid.to_numpy()
fitted_q1e = model_q1c_iB.fittedvalues.to_numpy()

In [124]:
print(np.round(residuals_q1e.sum(), 4))

print(np.round(y_q1e.mean(), 4))
print(np.round(fitted_q1e.mean(), 4))

Xt_resid_q1e = X_q1e.T @ residuals_q1e
print("X' u =", np.round(Xt_resid_q1e, 4))

corr_q1e = np.corrcoef(y_q1e, fitted_q1e)[0, 1]
r2_from_corr_q1e = corr_q1e**2

print('corr(y,yhat)^2:', r2_from_corr_q1e)
print('R^2 (statsmodels):', model_q1c_iB.rsquared)

-0.0
0.0
0.0
X' u = 0.0
corr(y,yhat)^2: 0.022891756621222583
R^2 (statsmodels): 0.022891756621222892


For this no-constant FWL regression, orthogonality to the included regressor and the corr-squared equals R-squared identity still hold, but the zero-sum residual and mean-matching properties are not guaranteed by construction.

# 2

## Question 2(a)(i)

In [125]:
y_q2a = analysis_df_q1q2['earn']
X_q2a = sm.add_constant(analysis_df_q1q2[['higrade', 'age', 'agesq']])

model_q2a_ur = sm.OLS(y_q2a, X_q2a).fit()
print(model_q2a_ur.summary())

                            OLS Regression Results                            
Dep. Variable:                   earn   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     102.5
Date:                Mon, 16 Feb 2026   Prob (F-statistic):           2.08e-65
Time:                        16:23:03   Log-Likelihood:                -91983.
No. Observations:               10940   AIC:                         1.840e+05
Df Residuals:                   10936   BIC:                         1.840e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1221.0501    186.247     -6.556      0.0

In [126]:
ser_q2a_i = np.sqrt(model_q2a_ur.ssr / model_q2a_ur.df_resid)
print('The standard error of the regression is', np.round(ser_q2a_i, 3))
print('The standard error of the education coefficient is', np.round(model_q2a_ur.bse['higrade'], 3))
print('The standard error of the age coefficient is', np.round(model_q2a_ur.bse['age'], 3))

The standard error of the regression is 1084.861
The standard error of the education coefficient is 6.606
The standard error of the age coefficient is 11.727


## Question 2(a)(ii)

Every additional year of schooling completed is associated with about $105.73 higher earnings, holding age and age squared fixed.

## Question 2(b)(i)

In [127]:
test_q2b_i = model_q2a_ur.t_test('higrade = 0')
print(test_q2b_i)

                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0           105.7343      6.606     16.007      0.000      92.786     118.683


## Question 2(b)(ii)

In [128]:
q_q2b_ii = 1
df2_q2b_ii = model_q2a_ur.df_resid

In [129]:
X_q2b_ii_r = sm.add_constant(analysis_df_q1q2[['age', 'agesq']])
model_q2b_ii_r = sm.OLS(y_q2a, X_q2b_ii_r).fit()

In [130]:
F_q2b_ii = ((model_q2a_ur.rsquared - model_q2b_ii_r.rsquared) / q_q2b_ii) / ((1 - model_q2a_ur.rsquared) / df2_q2b_ii)
p_q2b_ii = stats.f.sf(F_q2b_ii, q_q2b_ii, df2_q2b_ii)

In [131]:
print('UR R^2:', model_q2a_ur.rsquared)
print('R  R^2:', model_q2b_ii_r.rsquared)
print(f'F (R^2): {F_q2b_ii:.6f}, p-value: {p_q2b_ii:.6g}')

UR R^2: 0.0273378726082113
R  R^2: 0.004550279886713748
F (R^2): 256.209332, p-value: 5.09967e-57
